# ImageAnalyst
## 이미지 파일 로드 & 포즈 예측 & 자료구조화
- 정규표현식 사용
- 이미지의 이름으로 부터 ID와 ACTION, SEQUENCE를 추출
- 가령, `image478fall05.jpg`의 ID는 `478`, ACTION은 `fall`, SEQUENCE는 `5` 입니다. 
- 이미지를 불러와서 포즈 예측
- 모든 이미지의 예측된 포즈를 리스트에 저장

### 정규표현식 엔진 생성
- 파일 이름에서 숫자를 추출하는 표현식
- 파일 이름에서 문자열을 추출하는 표현식

In [1]:
import re
num_regex = re.compile(r'\d+')
action_regex = re.compile(r'\D+')

### 파일 이름 해석 함수들
- get_id : 파일 이름으로부터 id를 추출
- get_squence : 파일 이름으로부터 sequence 추출
- get_action : 파일 이름으로부터 action 추출
- `action_dict` : 액션 별로 숫자를 부여, ground truth가 됩니다.

In [2]:
action_dict = {'fall' : 0, 'no_fall' : 1,'walk' : 1, 'sleep' : 2, 'sit' : 3, 'bend' : 4}

get_id = lambda filename : int(num_regex.findall(filename)[0])

get_sequence = lambda filename : int(num_regex.findall(filename)[1])

def get_action(filename) :
    action = action_regex.findall(filename)[1]
    return action_dict[action]

### my_sort
- 파일들의 이름을 verbose(human-readable) 하게 정렬합니다.
- squence 순으로 정렬한 다음
- id 순으로 정렬함

In [3]:
def my_sort(l) :
    l.sort(key = get_sequence)
    l.sort(key = get_id)
    return l
imgname = 'image478fall05'
get_id(imgname), get_action(imgname), get_sequence(imgname)

(478, 0, 5)

### my_file_loader
- `data_folders`를 순차접근
- `data_folders` 원소들은 각각 액션의 자료를 담고 있음
- 존재하는 모든 파일의 이름을 하나의 리스트에 추합 : `files_as_action` 리스트
- `files_as_action` 리스트를 통해 모든 이미지 자료를 action, id, sequence 축으로 접근할 수 있다

In [4]:
data_folders = ['./data/Train Image/fall', './data/Train Image/no_fall']

In [5]:
import os

def my_file_loader() :
    
    files_as_action = list()
    for i, folder in enumerate(data_folders) :
        onlyfiles = my_sort(next(os.walk(folder))[2])
        min_id = get_id(onlyfiles[0])
        max_id = get_id(onlyfiles[-1])
        
        files_as_id = list()
        files_as_sequence = list()
        
        prev_id = min_id
        for file in onlyfiles :
            my_id = get_id(file)
            my_path = os.path.join(folder, file)
            if my_id != prev_id:
                files_as_id.append(files_as_sequence)
                files_as_sequence = [my_path]
                prev_id = my_id
            else :
                files_as_sequence.append(my_path)
                
        files_as_id.append(files_as_sequence) #맨 마지막에 누락되는 id를 빠뜨리지 않습니다
        files_as_action.append(files_as_id)
        
        print('파일이 로드되었습니다.')
        print('{}번 액션:"{}" {} ~ {}'.format(i, folder, min_id, max_id))
        
    return files_as_action

In [6]:
%time files_as_action = my_file_loader()

파일이 로드되었습니다.
0번 액션:"./data/Train Image/fall" 1 ~ 559
파일이 로드되었습니다.
1번 액션:"./data/Train Image/no_fall" 560 ~ 1063
CPU times: user 317 ms, sys: 31.7 ms, total: 348 ms
Wall time: 347 ms


- 맨 마지막 액션의, 맨 마지막 자료의 맨 마지막 시퀀스를 불러옵니다

In [7]:
files_as_action[-1][-1][-1]

'./data/Train Image/no_fall/image1063no_fall11.jpg'

### 오픈 포즈를 사용하여 포즈 예측 & 저장
- 이미지당 (2, 프레임 수,17, 2) 행렬 생성
- 배치가 2인 이유는 좌우반전한 좌표도 생성하기 때문임

In [8]:
from Skeleton.OneShotSkeleton import OneShotSkeleton
import cv2
from mxnet import nd, gpu
import pickle

ctx = gpu(1)

oss = OneShotSkeleton(ctx=ctx)
augument = True
bbox_thr = 0.6

x_train, y_train = [], []

for i, files_as_id in enumerate(files_as_action) :
    for j, files in enumerate(files_as_id) :
        coords, confidences = [], []
        aug_coords = []
        for file in files :
            frame = cv2.imread(file)
            frame = nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')
            width = frame.shape[1]
            pred_coords, _, bbox = oss(frame, bbox_thr=bbox_thr)
            coords.append(pred_coords[0])

            if augument :
                aug_coord = pred_coords[0].copyto(ctx)
                if len(bbox) > 0 :
                    aug_coord[:,0] = width - aug_coord[:,0]
                aug_coords.append(aug_coord)
        try :    
            coords = nd.stack(*coords)
            if augument :
                aug_coords = nd.stack(*aug_coords)
                ground_truth = nd.array([i]).as_in_context(ctx)
                print(i, j, coords.shape, aug_coords.shape, ground_truth)
                x_train.extend((coords, aug_coords))
                y_train.extend((ground_truth, ground_truth))
            else :
                x_train.extend(coords)
        except :
            print(file, "ignored")
    
    with open('./data_pkl/img_data_binary.pkl', 'wb') as f :
        pickle.dump(x_train, f)
        pickle.dump(y_train, f)

/home/aistudy/.conda/envs/python35/lib/python3.5/site-packages/mxnet/gluon/block.py:1159: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
0 0 (23, 17, 2) (23, 17, 2) 
[0.]
<NDArray 1 @gpu(1)>
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
0 1 (20, 17, 2) (20, 17, 2) 
[0.]
<NDArray 1 @gpu(1)>
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350, 3)
(350, 350,

KeyboardInterrupt: 